In [ ]:
from operator import mod
import numpy as np
import pulse2percept as p2p
%matplotlib inline
from matplotlib import pyplot as plt
from swarm_helpers import particle_swarm
from implant_helpers import build_implant

import pyswarms as ps
import pyswarms.backend as P
from pyswarms.backend.swarms import Swarm
from pyswarm import pso
import json

In [ ]:
# Create model and implant
# x_range = model.spatial.retinotopy.ret2dva(-2500, 2000)
model = p2p.models.AxonMapModel()
x_range = model.spatial.retinotopy.ret2dva(-2500, 2000)
y_range = model.spatial.retinotopy.ret2dva(-2000, 2000)

model = p2p.models.AxonMapModel(xrange=x_range, yrange=y_range, xystep=0.5, rho=100)
model.build()

# Bounds 
bound_params = {
  'x': (-2500, 2000),
  'y': (-2000, 2000)
}
o_lb = [v[0] for v in bound_params.values()]
o_ub = [v[1] for v in bound_params.values()]
    
overlap_bounds = (o_lb, o_ub)


In [ ]:
NUM_ITERATIONS = 100
NUM_PARTICLES = 100

data = open('data/data.json')
experiments = json.load(data)
data.close()

for num_electrodes in experiments:
    for electrode_size in experiments[num_electrodes]:
        for loss_func in experiments[num_electrodes][electrode_size]:
            
            print(f"---------Experiment---------")
            print(f"Number of Electrodes: {num_electrodes}\nElectrode Size: {electrode_size} microns\nLoss Function: {loss_func}")
            print(f"Number of Iterations: {NUM_ITERATIONS}\nNumber of Particles: {NUM_PARTICLES}")
            
            search_params = {}
            for electrode in range(int(num_electrodes)):
            
                search_params["x" + str(electrode)] = (-2500, 2000)
                search_params["y" + str(electrode)] = (-2000, 2000)

            lb = [v[0] for v in search_params.values()]
            ub = [v[1] for v in search_params.values()]

            bounds = (lb, ub)

            # Maximize number of effective electrodes
            best_positions, best_cost, n_eff = particle_swarm(num_electrodes, electrode_size, NUM_PARTICLES, NUM_ITERATIONS, bounds, overlap_bounds, model, experiments, loss_func) 
            
            print(f"Best Cost: {best_cost}")
            print(f"Best Number Effective: {n_eff}")
            print(f"----------------------------")
            print()


In [ ]:
with open('data/trial1.json', 'r') as data:
    experiments = json.load(data)
    best_effective = []
    best_positions = []

    for num_electrodes in experiments:
        best_num_eff = 0
        for electrode_size in experiments[num_electrodes]:
            for loss_func in experiments[num_electrodes][electrode_size]: 
                if experiments[num_electrodes][electrode_size][loss_func]["best_num_eff"] >= best_num_eff:
                    if num_electrodes == "60" and electrode_size == "100":
                        best_num_eff = experiments[num_electrodes][electrode_size][loss_func]["best_num_eff"]
                        best_pos = experiments[num_electrodes][electrode_size][loss_func]["best_positions"]
                        # best_losses[num_electrodes].append(loss_func)
                        # print("------------")
                        print(num_electrodes, electrode_size, loss_func, experiments[num_electrodes][electrode_size][loss_func]["initial_num_effective"])
                   

                        best_effective.append(best_num_eff)
                        best_positions.append(best_pos)
        
    for index in range(len(best_effective)):
        print("Best num:", best_effective[index])
    # print(len(best_positions))
    # best_implant = build_implant(best_positions[4])
    # best_implant.plot()
    # best_implant
    # model.plot()

In [ ]:
best_implant = build_implant(best_positions[0])
best_implant.plot()
best_implant
model.plot()